<a href="https://colab.research.google.com/github/ucfilho/raianars_fly/blob/master/RTC_DE_desmonta_IF_new_16_mar_03_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from scipy.optimize import differential_evolution
import numpy as np

In [0]:
# Funcoes utilizadas neste codigo
# Area(U,tempAtual,tempInicial,...): constroi matriz de areas trocadores 
#                                    entra duas correntes sendo chamada no loop
# buildMatriz(x,cols): constroi a matriz deltaT e binaria (logica de trocas)
#
# Temperatura(matrizDeltaT,...): constroi as temperaturas e cargas termicas

In [0]:
def buildMatriz(x,cols):

  nref=int(len(x)/2) # metade de x é deltaT e a outra metade variavel logica
  matrizDeltaT=np.zeros((cols,cols))
  matrizBinaria=np.zeros((cols,cols))

  contador=0
  for i in range(cols):
    for j in range(cols):
      soma=contador+nref
      #print('%d %d %d %d %d'%(i,j,contador,nref,soma))
      flag=0
      if(i<j):
        if(contador<nref):
          flag=1
      if(flag>0):
        matrizDeltaT[i,j]=x[contador]
        matrizBinaria[i,j]=round(x[contador+nref])
        contador=contador+1
        
  return matrizDeltaT,matrizBinaria

In [0]:
def Temperatura_Carga(mCp,matrizDeltaT,matrizBinaria,tempInicial):
  global UsoFuncao

  cols=len(mCp)
  cargaTermica=np.zeros((cols,cols))
  contador=0
  cont_uso=0
  tempAtual=np.copy(tempInicial)
  erro=0 # valor erro se torna erro=1 quando ha algum erro e vai penalizar
  for i in range(cols):
    for j in range(cols):
      contador=contador+1
      cargaTermica[i,j]=mCp[i]*matrizDeltaT[i,j]

      if(i<j):
        if(matrizBinaria[i,j]==1):
          #i: CQ, j: CF
          #tempSaidaQ=tempAtual[i]+matrizDeltaT[i,j]
          tempSaidaQ=tempInicial[i]+matrizDeltaT[i,j]
          tempSaidaF=-cargaTermica[i,j]/(mCp[j])+tempAtual[j]
          tempSaidaF=-cargaTermica[i,j]/(mCp[j])+tempInicial[j]
          flag=0 # flag : indica que nao entra no teste logico
          # usei flag no lugar q ifs pq a identacao ficou melhor assim
          if(tempAtual[i] > tempAtual[j]): #Tquente maior Tfria: vaores atuais
            if(tempSaidaQ > tempSaidaF): #Tquente maior Tfria: saidas
              if(contador < cols):
                flag=1
          
          if(flag>0):
            tempAtual[i]=tempSaidaQ
            tempAtual[j]=tempSaidaF
            cont_uso=cont_uso+1
            #print(cont_uso)
            UsoFuncao.append(cont_uso)
          else:
            erro=1


  return tempAtual,cargaTermica,erro

In [0]:
def Area(U,tempAtual,tempInicial,cargaTermica,matrizDeltaT):
  global UsoFuncao

  cols=len(tempInicial)
  Trocador=np.zeros((cols,cols)) # matriz q armazena as areas dos trocadores
  Area=[]

  for i in range(cols):
    for j in range(cols):
      if(i<j):
        Carga=cargaTermica[i,j]
        # calcula a area de placas das duas correntes que trocam i e j

        #a corrente i perde calor para a j
        #i: CQ, j:CF
        tempSaidaF=tempAtual[j]
        tempSaidaQ=tempAtual[i]
        deltaT1=tempInicial[i]-tempSaidaF
        deltaT2=tempSaidaQ-tempInicial[j]
        try:
          A=(deltaT1-deltaT2)
          B=np.log((deltaT1/deltaT2))
          deltaTMediaLog=A/B
        except:
          deltaTMediaLog=1e99
 
        if(deltaTMediaLog==1e99):  
          AreaCalc=1e99
        else:
          AreaCalc=abs(Carga/(U*deltaTMediaLog))
        
        if(tempAtual[i]==tempInicial[i]):
          AreaCalc=0

        if(tempAtual[j]==tempInicial[j]):
          AreaCalc=0

        ''''''
        Area.append(AreaCalc)

  
  return Area

In [0]:
def Utilidades(tempAlvo,tempAtual,mCp):

  cols=len(mCp)
  cargaUQ=0
  cargaUF=0
  
  '''
  caso I cita TUQin=553 e TQout=552, TUFin=303 TUFout=353
  '''
  
  TUQin=553
  TUQout=552
  TUFin=303 
  TUFout=353
  areaTrocador=[]

  for j in range(cols): 
    delta=tempAlvo[j]-tempAtual[j]
    if(delta>0):#usa UQ
      cargaUQ=cargaUQ+mCp[j]*delta
      deltaT1=(TUQin-tempAlvo[j])
      deltaT2=(TUQout-tempAtual[j])

      try:
        A=deltaT1-deltaT2
        B=np.log(deltaT1/deltaT2)
        deltaTMediaLog=A/B
      except:
        deltaTMediaLog=1e99

      if(deltaTMediaLog==1e99):
        areaTrocador.append(1e99)
      else:
        areaTrocador.append((abs(mCp[j]*(delta)/(U*deltaTMediaLog))))

    elif(delta==0):#nao usa UF e UQ
      deltaTMediaLog=0
      areaTrocador.append(0) 

    else:#usa UF
      deltaT1=(tempAtual[j]-TUFout)
      deltaT2=(tempAlvo[j]-TUFin)     
      cargaUF=cargaUF+mCp[j]*(-delta)

      #UF -> água resfriada -> entra a 303K e sai a 353K
      try:
        A=deltaT1-deltaT2
        B=np.log(deltaT1/deltaT2)
        deltaTMediaLog=A/B
      except:
        deltaTMediaLog=1e99
      
      if(deltaTMediaLog==1e99):
        areaTrocador.append(1e99)
      else:
        areaTrocador.append((abs(mCp[j]*(-delta)/(U*deltaTMediaLog))))

  return areaTrocador, cargaUF,cargaUQ 
  # areaTrocador contem a soma das areas
  # Trocadores a area associada a cada corrente
  # cargaUF e cargaUQ kW gasto

In [0]:
def CustoProcesso(areaTrocadores,areaUtilidades,cargaUF,cargaUQ):
  global coefCustoUQ,coefCustoUF
  costTrocadores=0
  for it in areaTrocadores:
    for wii in areaUtilidade:
      if(it<1e99):
        if(wii<1e99):
          costTrocadores=costTrocadores+300*(it**0.5+wii**0.5)
        else:
          costTrocadores=1e99
          break
        
  costUQ=coefCustoUQ*cargaUQ
  costUF=coefCustoUF*cargaUF

  if(costTrocadores==1e99):
    cost=1e99
  else:
    cost=costUQ+costUF+costTrocadores
  return cost

In [0]:
#FUN calcula o TAC para um determinado arranjo
#  cada calculo é feito em uma funcao a parte Fun usa todas funcoes 

def FUN(x):

  # global mCp,tempAtual,tempAlvo,U,coefCustoUQ,coefCustoUF
  global mCp,tempAtual,tempAlvo,tempInicial,U,coefCustoUQ,coefCustoUF
  global UsoFuncao


  cols=len(mCp)
  #tempInicial=np.copy(tempAtual) 
  matrizDeltaT,matrizBinaria=buildMatriz(x,cols) # transforma x em matriz logica e troca
  tempAtual,Carga,Erro=Temperatura_Carga(mCp,matrizDeltaT,matrizBinaria,tempInicial)
  areaTrocadores =Area(U,tempAtual,tempInicial,Carga,matrizDeltaT)
  areaUtilidade, cargaUF,cargaUQ = Utilidades(tempAlvo,tempAtual,mCp)
  #print('estou aqui');print(areaUtilidade);print(areaTrocadores)
  #custoTotal=CustoProcesso(areaTrocadores,areaUtilidade,cargaUF,cargaUQ)
  #print('Custo total');print(custoTotal);
  cost=CustoProcesso(areaTrocadores,areaUtilidade,cargaUF,cargaUQ)
  #print('Custo processo');print(cost);
  funr = np.where(np.isnan(cost), 1e99, cost)

  if(Erro>0):
    funr=1e99


  return funr

In [0]:
#MATGRAF retorna o gráfico das trocas térmicas em cada trocador
# function matrizGrafico=MATGRAF(matrizDeltaT,mCp,tempAtual)
def MATGRAF(matrizDeltaT,mCp,tempAtual):
  cols=len(mCp)
  contador=0
  matrizGrafico=[]
  matrizGrafico.append(tempAtual)
  for i in range(cols):
    for j in range(cols):
      if(i<j):
        contador=contador+1
        # i: CQ, j:CF
        cargaTermica=mCp[i]*matrizDeltaT[i,j]
        tempSaidaQ=tempAtual[i]+matrizDeltaT[i,j]
        tempSaidaF=-cargaTermica/(mCp[j])+tempAtual[j]

        flag=0 # flag : indica que nao entra no teste logico
        # usei flag no lugar ifs pq identacao ficou melhor 
        if(tempAtual[i] > tempAtual[j]): #Tquente maior Tfria: valores atuais
          if(tempSaidaQ > tempSaidaF): #Tquente maior Tfria: saidas
            if(contador < cols):
              flag=1
        

        if(flag>0):
          tempAtual[i]=tempSaidaQ
          tempAtual[j]=tempSaidaF
          matrizGrafico.append(tempAtual)
          
  df=pd.DataFrame(matrizGrafico)
  
  return df

In [0]:
#MATBEST retorna a matriz com os deltas das trocas térmicas que ocorreram e a matriz binária
# function [matrizTrocaBest,matrizBinaria]=MATBEST(x,mCp,tempAtual)
def MATBEST(x,mCp,tempAtual):
  
  nref=int(len(x)/2)
  #nref=0
  cols=len(mCp)

  # cada particula contem metade das variaveis para matriz binaria
  # e outra metade para deltaT use nref e para separar os dois casos
  matrizDeltaT=np.zeros((cols,cols))
  matrizTrocaBest=np.zeros((cols,cols))
  matrizBinaria=np.zeros((cols,cols))

  contador=0
  for i in range(cols):
    for j in range(cols):
      if(i<j):
        matrizDeltaT[i,j]=x[contador]
        matrizBinaria[i,j]=round(x[contador+nref])
        contador=contador+1
  
    for i in range(cols):
      for j in range(cols):
        if(i<j):
          if(matrizBinaria[i,j]==1):
            #i: CQ, j: CF
            cargaTermica=mCp[i]*matrizDeltaT[i,j]
            tempSaidaQ=tempAtual[i]+matrizDeltaT[i,j]
            tempSaidaF=-cargaTermica/(mCp[j])+tempAtual[j]
            if(
                  tempAtual[i]>tempAtual[j] and
                  tempSaidaQ>tempAtual[j] and
                  tempAtual[i]>tempSaidaF
              ): #TQentra>TFentra & TQsai>TFentra & TQentra>TFsai
              tempAtual[i]=tempSaidaQ
              tempAtual[j]=tempSaidaF
            else:
              matrizDeltaT[i,j]=0
  matrizTrocaBest=matrizDeltaT
  return matrizTrocaBest,matrizBinaria

In [0]:
#********************PROGRAMA PRINCIPAL*************************

NPAR=50 #Número de partículas
ITE=5000 #Número de iterações
PAR=12 #Número de parâmetros a ser otimizados

MAX=[0,0,0,0,0,0,1,1,1,1,1,1]
MIN=[-300,-300,-300,-300,-300,-300,0,0,0,0,0,0]
mCp=[4,2,1.5,3] #[kW/K]
tempInicial=[453,393,523,533] #[K]
#tempAtual=[453,393,523,533] #[K]
tempAlvo=[513,508,403,433] #[K]
U=0.2 #[kW/(m2.K)]

coefCustoUQ=110 #[$/(kW.ano)]
coefCustoUF=12.2 #[$/(kW.ano)]

bounds = []
for i in range(12):
  bounds.append((MAX[i],MIN[i]))



In [12]:
cols=len(mCp)
UsoFuncao=[]
x=[-99.214722,59.99994,146.225,4.2108538,15.99999,0, 0.9500049,1, 1,0.9103451, 1,0.4786627]
matrizDeltaT,matrizBinaria=buildMatriz(x,cols) # transforma x em matriz logica e troca
tempAtual,Carga,Erro=Temperatura_Carga(mCp,matrizDeltaT,matrizBinaria,tempInicial)
areaTrocadores =Area(U,tempAtual,tempInicial,Carga,matrizDeltaT)
areaUtilidade, cargaUF,cargaUQ = Utilidades(tempAlvo,tempAtual,mCp)
print('estou aqui');print(areaUtilidade);print(areaTrocadores)
custoTotal=CustoProcesso(areaTrocadores,areaUtilidade,cargaUF,cargaUQ)
print('Custo total');print(custoTotal);
cost=CustoProcesso(areaTrocadores,areaUtilidade,cargaUF,cargaUQ)
print('Custo processo');print(cost);
print('cont_uso');print(UsoFuncao)

estou aqui
[18.432008054657363, 12.733140081731566, 6.822363227942191, 9.762672013038838]
[0, 0, 0, 0, 0, 0]
Custo total
82032.57443684274
Custo processo
82032.57443684274
cont_uso
[]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide


In [13]:
UsoFuncao=[]
fchoice=FUN
result = differential_evolution(fchoice, bounds, maxiter=ITE, popsize=NPAR)
print('temperatura atual=',tempAtual)
print(result.x, result.fun)
print(tempAtual)
GBEST=result.x
BEST=result.fun
matrizTrocaBest,matrizBinaria=MATBEST(GBEST,mCp,tempAtual)
matrizGrafico=MATGRAF(matrizTrocaBest,mCp,tempAtual)

print('Melhor solução para %d iterações com %d partículas \n'%(ITE,NPAR));print(GBEST)
print('Valor da função f(gbest)=%f \n'%BEST)
print('matriz grafico');print(matrizGrafico)
print('cont_uso');print(UsoFuncao)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide


temperatura atual= [452 394 523 533]
[-7.18435258e-01 -5.26352212e+00 -3.23636284e+01 -2.48070720e+02
 -1.47626152e+02 -1.11266045e+02  5.19450875e-01  1.75649513e-01
  4.47316969e-01  1.80204110e-01  7.12111694e-02  1.61512748e-01] 78173.14536403562
[452 394 523 533]
Melhor solução para 5000 iterações com 50 partículas 

[-7.18435258e-01 -5.26352212e+00 -3.23636284e+01 -2.48070720e+02
 -1.47626152e+02 -1.11266045e+02  5.19450875e-01  1.75649513e-01
  4.47316969e-01  1.80204110e-01  7.12111694e-02  1.61512748e-01]
Valor da função f(gbest)=78173.145364 

matriz grafico
     0    1    2    3
0  447  399  523  533
1  447  399  523  533
cont_uso
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [14]:
cols=len(mCp)
matrizDeltaT,matrizBinaria=buildMatriz(GBEST,cols) # transforma x em matriz logica e troca
tempAtual,cargaTermica,erro=Temperatura_Carga(mCp,matrizDeltaT,matrizBinaria,tempInicial)
areaTrocadores =Area(U,tempAtual,tempInicial,cargaTermica,matrizDeltaT)

print('temperatura inicial');print(tempInicial)
print('temperatura alvo');print(tempAlvo)
print('temperatura atual');print(tempAtual)
print('Area trocadores');print(areaTrocadores)

temperatura inicial
[453, 393, 523, 533]
temperatura alvo
[513, 508, 403, 433]
temperatura atual
[452 394 523 533]
Area trocadores
[nan, 0, 0, 0, 0, 0]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide


In [15]:
print(cols)
print(GBEST)

4
[-7.18435258e-01 -5.26352212e+00 -3.23636284e+01 -2.48070720e+02
 -1.47626152e+02 -1.11266045e+02  5.19450875e-01  1.75649513e-01
  4.47316969e-01  1.80204110e-01  7.12111694e-02  1.61512748e-01]


In [16]:
matrizDeltaT,matrizBinaria=buildMatriz(GBEST,cols) # transforma x em matriz logica e troca
print('Matriz de Trocas Térmicas \n')
print(matrizDeltaT)
print('Matriz Binária \n')
print(matrizBinaria)

Matriz de Trocas Térmicas 

[[   0.           -0.71843526   -5.26352212  -32.36362839]
 [   0.            0.         -248.07072036 -147.62615174]
 [   0.            0.            0.         -111.26604535]
 [   0.            0.            0.            0.        ]]
Matriz Binária 

[[0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
